In [1]:
# Comparison Notebook

In [2]:
import cudf
import cugraph

import pandas as pd
import networkx as nx

In [3]:
%%timeit
gdf_tw_edges = cudf.read_csv("07a.Tw.edges.csv")

146 ms ± 689 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
#%%timeit
#pdf_tw_edges = pd.read_csv("07a.Tw.edges.csv")

In [5]:
gdf_tw_edges = cudf.read_csv("07a.Tw.edges.csv")
G = cugraph.Graph(directed=False)
G.from_cudf_edgelist(gdf_tw_edges, source='from', destination='to', renumber=True)
(G.number_of_nodes(), G.number_of_edges())
degree = G.degrees()
degreeP = degree.to_pandas()

In [6]:
degree = G.degrees()
degreeP = degree.to_pandas()

In [7]:
%%timeit
ds = degree.sort_values(by='out_degree', ascending=False)

6.54 ms ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
dps = degreeP.sort_values(by='out_degree', ascending=False)

205 ms ± 2.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
comp = cugraph.weakly_connected_components(G)
label_count = comp.groupby('labels').count()
label_count.rename(columns={"vertex": "count"}, inplace=True)

print("Total number of components found : ", "{:,}".format(len(label_count)))

Total number of components found :  107,493


In [10]:
# Get the label ID for the largest component
max_comp = label_count['count'].max()
big_comp_id = label_count[label_count['count'] == max_comp]

# need to reset the index since the groupby moved labels to be the index
big_comp_id.reset_index(drop=False, inplace=True)
id = big_comp_id['labels'][0]

# now need to get the vertex IDs associated with those good components
big_comp = comp[comp['labels'] == id]

# Now get the subgraph that just contains those nodes in large compoents
subgraph = cugraph.subgraph(G, big_comp['vertex'])

(subgraph.number_of_nodes(), subgraph.number_of_edges())

(2668807, 5741055)

In [11]:
pdf_edge_list = subgraph.to_pandas_edgelist()

In [14]:
Gnx = nx.from_pandas_edgelist(pdf_edge_list, 'src', 'dst')

In [ ]:
import networkx.algorithms.community as nx_comm

In [19]:
%%timeit
_ = nx_comm.louvain_communities(Gnx)

7min 49s ± 33.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
communities, mod_score = cugraph.louvain(subgraph)

865 ms ± 1.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


-----
Louvain on full graph

In [21]:
%%timeit
communities, mod_score = cugraph.louvain(G)

773 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
pdf = gdf_tw_edges.to_pandas()

In [26]:
%%time
Gnx = nx.from_pandas_edgelist(pdf, 'from', 'to')

CPU times: user 28.7 s, sys: 2.16 s, total: 30.9 s
Wall time: 30.8 s


In [27]:
%%timeit
communities, mod_score = cugraph.louvain(G)

769 ms ± 2.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit
_ = nx_comm.louvain_communities(Gnx)

8min 37s ± 45.8 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
